# ML Based Sentiment Analysis

## Imports

In [2]:
import nltk


# file paths

In [3]:
pos_reviews_path = "rt-polaritydata/rt-polaritydata/rt-polarity.pos"
neg_reviews_path = "rt-polaritydata/rt-polaritydata/rt-polarity.neg"

### Constants

In [6]:
split_index = 2500

## Opening and reading lines

In [4]:
with open(pos_reviews_path, 'r', encoding='latin-1') as pos_file :
    pos_reviews = pos_file.readlines()

In [5]:
with open(neg_reviews_path, 'r', encoding='latin-1') as neg_file :
    neg_reviews = neg_file.readlines()

### Using Naive Bayes from NLTK 

In [8]:
test_negative_reviews = neg_reviews[split_index+1:]
test_positive_reviews = pos_reviews[split_index+1:]
training_negative_reviews = neg_reviews[:split_index]
training_positive_reviews = pos_reviews[:split_index]

In [9]:
def get_vocabulary() :
    pos_word_list = [word for line in training_positive_reviews for word in line.split()]
    neg_word_list = [word for line in training_negative_reviews for word in line.split()]
    all_word_list = [item for sublist in [pos_word_list, neg_word_list] for item in sublist]
    all_word_set = list(set(all_word_list))
    vocabulary = all_word_set
    return vocabulary
    

In [10]:
vocabulary = get_vocabulary()

In [13]:
len(vocabulary)

14094

In [18]:
def get_training_data():
    neg_train_tagged_list = [{'review': one_review.split(), 'label' : 'negative'} for one_review in training_negative_reviews]
    pos_train_tagged_list = [{'review': one_review.split(), 'label' : 'positive'} for one_review in training_positive_reviews]
    full_tagged_list = [item for sublist in [neg_train_tagged_list, pos_train_tagged_list] for item in sublist]
    training_data = [(review['review'],review['label']) for review in full_tagged_list]
    return training_data
    

In [19]:
training_data = get_training_data()

In [20]:
training_data[0]

(['simplistic', ',', 'silly', 'and', 'tedious', '.'], 'negative')

In [21]:
len(training_data)

5000